In [ ]:
# Python 2 & 3 Compatibility
from __future__ import print_function, division

In [ ]:
import matplotlib
% matplotlib inline

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt

import matplotlib.pyplot as plt

# %matplotlib inline

In [ ]:
from bs4 import BeautifulSoup
import requests
import time

In [ ]:
# [str(i) for i in range(2016,1979,-1)]

In [ ]:
mvYRlist = [str(i) for i in range(2016,1979,-1)]

In [ ]:
len(mvYRlist)

In [ ]:
# http://www.boxofficemojo.com/yearly/chart/?yr=2016&p=.htm
# http://www.boxofficemojo.com/yearly/chart/?yr=####&p=.htm

# http://www.boxofficemojo.com/yearly/chart/?page=1&view=releasedate&view2=domestic&yr=2016&p=.htm    

bomjURL='http://www.boxofficemojo.com'
YRpg1viewerURL = '/yearly/chart/?page=1&view=releasedate&view2=domestic&yr='
YRpg1endURL = '&p=.htm'

YRpg1URLlist=[(bomjURL+YRpg1viewerURL+year+YRpg1endURL) for year in mvYRlist]

In [ ]:
len(YRpg1URLlist)

In [ ]:
YRpg1URLlist

In [ ]:
# pgList=[]

In [ ]:
# PGheaders = ['pgURL', 'startPg', 'endPg']
pgList=[]
YstartendPages=[]

for y in range(0, len(YRpg1URLlist)):
    url=YRpg1URLlist[y]
    #print(url)
    
    YstartendPages.append([mvYRlist[y],1,100])
    #pg1URL = url
    #pg1S = 1
    #pg1E = 100
    
    #pgList.append((url,'1','100'))
    #pgList1_dict = dict(zip(PGheaders, [pg1URL, pg1S, pg1E]))
    #pgList.append(pgList1_dict)
    
    pgList.append(url)
    
    PG1resp = requests.get(url)
    if PG1resp.status_code==200:
        PG1soup = BeautifulSoup(PG1resp.text,'lxml')
        
    pg1_tmp = PG1soup.find_all('center')[0].find_all('a')
    
    #     pgList = [ (bomjURL+p['href']) 
    for p in pg1_tmp:
        pgURLs = (bomjURL+p['href'])
        #print(pgURLs)
        
        pg_rangeSTR = p.text.replace('#','').replace('\x96','')
        pgS, pgE = (int(pg_rangeSTR[0:3]), int(pg_rangeSTR[3:7]) )
        #print(pgS, pgE)
        YstartendPages.append( (mvYRlist[y],pgS,pgE) )
        
        #pgList.append((bomjURL+p['href'],pgS, pgE))
        #pgList_dict = dict(zip(PGheaders, [pgURLs, pgS, pgE]))    
        #pgList.append(pgList_dict)
        
        pgList.append(pgURLs)
              
    #break

In [ ]:
mvYRlist , YstartendPages

In [ ]:
len(pgList) == len(YstartendPages) , len(YstartendPages)

In [ ]:
pgList

In [ ]:
# mv_list=[]
pgList

In [ ]:
### to try to scrap table data from list of urls with start and end pg rowNos

In [ ]:
mv_list=[]

headers = ['mYrank','mID', 'mName', 'mStudio', 
           'totalDGross', 'totalDTheatre', 'openDGross','openDTheatre', 'openWkdate', 'closeWkdate']


mvCnt=0

for y in range(0, len(YstartendPages)):
    
    Year,pgSn,pgEn = YstartendPages[y]
    if (Year,pgSn,pgEn) == ('2016',501,530):
        pgSn=502   
    else:
        pass
    
    mvChartNrows=pgEn-pgSn+1
    mvCnt += mvChartNrows
    
    print('Retrieving ' + str(mvChartNrows) + 'rows from: ' + pgList[y])
    pgResp = requests.get(pgList[y])
    if pgResp.status_code == 200:
        chartpgSoup = BeautifulSoup(pgResp.text,'lxml')
    if int(Year) <= 2001:
        tableInfo=chartpgSoup.find_all('table')[3].find_all('font')[13:(mvChartNrows*8)+13] ## 8 lines for each table columns *N# mvrows on pg 
   
    else:
        tableInfo=chartpgSoup.find_all('table')[3].find_all('font')[14:(mvChartNrows*9)+14] ## 9 lines for each table columns *N# mvrows on pg 
    

    for i in range(1,mvChartNrows+1): #(1,3): #
        if int(Year) <= 2001:  
            mvinfo = tableInfo[8*(i-1):8*i]
        else:
            mvinfo = tableInfo[9*(i-1):9*i]
    
        mvYRrank = mvinfo[0].text
        
        if mvinfo[1].find('a') is None: 
            mvID = mvinfo[1].text
        else:
            mvID = mvinfo[1].find('a')['href'][12:-4]
        
        mvName = mvinfo[1].text
        
        pdtStudio = mvinfo[2].find('a')['href'][22:-4]
        
        totDGross = int(mvinfo[3].text.replace('$','').replace(',',''))
        
        if mvinfo[4].text!='N/A':
            totDTheatre = int(mvinfo[4].text.replace(',',''))
        else:
            totDTheatre = np.nan

        if mvinfo[5].text!='N/A':
            oDGross = int(mvinfo[5].text.replace('$','').replace(',',''))
        else:
            oDGross = np.nan
            
        if mvinfo[6].text!='N/A':
            oDTheatre = int(mvinfo[6].text.replace(',',''))
        else:
            oDTheatre = np.nan            
    
        if mvinfo[7].text!='-' and mvinfo[7].text!='Jan':
            oWkmd = dt.datetime.strptime((Year + '/' + mvinfo[7].text),'%Y/%m/%d')
            #dt.datetime.strptime(Year + '/' + mvinfo[7].find('a').text,'%m/%d')
        else:
            oWkmd = np.nan
    
        if int(Year) > 2001:
            if mvinfo[8].text!='-' and mvinfo[8].text != '2/29' and mvinfo[8].text !='2':
                cWkmd = dt.datetime.strptime((Year + '/' + mvinfo[8].text),'%Y/%m/%d')
                #dt.datetime.strptime(mvinfo[8].find('a').text,'%m/%d')
            else:
                cWkmd = np.nan
        else:
            cWkmd = np.nan
    
        movie_dict = dict(zip(headers, [mvYRrank, mvID, mvName, pdtStudio,
                                    totDGross, totDTheatre,
                                    oDGross, oDTheatre,
                                    oWkmd, cWkmd]))
        mv_list.append(movie_dict)
        
        #time.sleep(2)

In [ ]:
len(mv_list) , mvCnt

In [ ]:
mv_listDF = pd.DataFrame(mv_list, columns=headers)

In [ ]:
mv_listDF.to_csv('mv_chartListDF_2016_1980.csv')

In [ ]:
mv_listDF=pd.read_csv('mv_chartListDF_2016_1980.csv')

In [ ]:
mv_listDF.dtypes

In [ ]:
# mv_listDF.mName.sort_values() #.sort_values(ascending=False).reset_index().head(500)

In [ ]:
mv_listDF.mID.sort_values().reset_index().iloc

In [ ]:
# PGresponse = requests.get("http://www.boxofficemojo.com/yearly/chart/?page=1&view=releasedate&view2=domestic&yr=2016&p=.htm")
# PGresponse = requests.get("http://www.boxofficemojo.com/yearly/chart/?page=2&view=releasedate&view2=domestic&yr=2016&p=.htm")

In [ ]:
# PGresponse.status_code

In [ ]:
# soup = BeautifulSoup(PGresponse.text,'lxml')

#view-source:https://en.wikipedia.org/wiki/List_of_highest-grossing_films
#line92

# movie_list = soup.find("table",{ "class" : "wikitable sortable plainrowheaders" })
# print(movie_list)

In [ ]:
# pg_tmp = soup.find_all('center')[0].find_all('a')

In [ ]:
# pg_range = 
# pg_tmp[0].text.replace('#','').replace('\x96','')

In [ ]:
# pg_rangeSTR = pg_tmp[0].text.replace('#','').replace('\x96','')
# pgS,pgE = (pg_rangeSTR[:3],pg_rangeSTR[3:])
# # pgRange
# pgS, pgE

In [ ]:
# soup.find_all('center')[0].find_all('a')[0]['href']
# pgList = [ p['href'] for p in pg_tmp]

In [ ]:
# pgList

In [ ]:
# Rank, MovieID, MovieTitle, Studio, TotalDomGross, TDG_Theaters, OpeningGross, OG_Theaters, Open, Close

In [ ]:
# tableInfo=soup.find_all('table')[3].find_all('font')[14:914]## 9 lines for each of the table columns *100 rows

In [ ]:
# tableInfo

In [ ]:
# tabcolN=9
# mv_list=[]

# headers = ['mNo','mID', 'mName', 'mStudio', 
#            'totalDGross', 'totalDTheatre', 
#            'openDGross','openDTheatre', 
#            'openWkdate', 'closeWkdate']

# Year='2016'

# for i in range(1,101): #(1,3): #
#     mvinfo = tableInfo[9*(i-1):9*i]
    
#     mvN = mvinfo[0].text
#     mvID = mvinfo[1].find('a')['href'][12:-4]
#     mvName = mvinfo[1].text
#     pdtStudio = mvinfo[2].find('a')['href'][22:-4]
#     totDGross = int(mvinfo[3].text.replace('$','').replace(',',''))
#     totDTheatre = int(mvinfo[4].text.replace(',',''))
#     oDGross = int(mvinfo[5].text.replace('$','').replace(',',''))
#     oDTheatre = int(mvinfo[6].text.replace(',',''))
    
#     if mvinfo[7].text!='-':
#         oWkmd = dt.datetime.strptime((Year + '/' + mvinfo[7].text),'%Y/%m/%d')
#         #dt.datetime.strptime(Year + '/' + mvinfo[7].find('a').text,'%m/%d')
#     else:
#         oWkmd = np.nan
    
#     if mvinfo[8].text!='-':
#         cWkmd = dt.datetime.strptime((Year + '/' + mvinfo[8].text),'%Y/%m/%d')
#         #dt.datetime.strptime(mvinfo[8].find('a').text,'%m/%d')
#     else:
#         cWkmd = np.nan
    
    
#     movie_dict = dict(zip(headers, [mvN, mvID, mvName, pdtStudio,
#                                     totDGross, totDTheatre,
#                                     oDGross, oDTheatre,
#                                     oWkmd, cWkmd]))
#     mv_list.append(movie_dict)

In [ ]:
# headers = ['movie title',
#            'release date', 'runtime (mins)', 'rating' , 'production budget', 'domestic total gross']

# movie_data1 = []
# movie_dict = dict(zip(headers, [title,                                
#                                 release_date,
#                                 runtime,
#                                 rating,
#                                 prdbudget,
#                                 domestic_total_gross]))
# movie_data1.append(movie_dict)

# pprint(movie_data1)

In [ ]:
# pd.isnull((np.nan)) #True

In [ ]:
# soup.find_all('table')[3].find_all('a')
## find all href that has /movies/

In [ ]:
import pandas as pd

test=pd.read_html('http://www.the-numbers.com/movie/records/Biggest-Domestic-Weekends')

In [ ]:
test[0].columns = ['Rank','Date','mvName','Gross','Theaters','PerTheater','TotGross','Days']

In [ ]:
df = test[0]

In [ ]:
df.dtypes

In [ ]:
import datetime

df['Date']=pd.to_datetime(df.Date)

In [ ]:
df['mvName'] = df.mvName.apply(lambda x: str(x))

In [ ]:
df['Gross'] = df.Gross.apply(lambda x: int(x.replace('$','').replace(',','')))

In [ ]:
df['PerTheater'] = df.PerTheater.apply(lambda x: int(x.replace('$','').replace(',','')))

In [ ]:
df['TotGross'] = df.TotGross.apply(lambda x: int(x.replace('$','').replace(',','')))

In [ ]:
df.dtypes

In [ ]:
# del df2

In [ ]:
# df

In [ ]:
# df2=df.drop_duplicates(['mvName'])

In [ ]:
# df2

In [ ]:
# test[0][test[0].notnull()]

# nms.dropna(thresh=2)
tmp=pd.read_html('http://www.the-numbers.com/movie/budgets/all')
tmp0=tmp[0].dropna(thresh=2)[1:]
budgetPD=tmp0.iloc[:,1:6]

In [ ]:
budgetPD.columns = ['ReleaseDate', 'mvName', 'ProdBudget', 'DGross', 'WwGross']

In [ ]:
import datetime
budgetPD['ReleaseDate']=pd.to_datetime(budgetPD.ReleaseDate)

In [ ]:
budgetPD['mvName'] = budgetPD.mvName.apply(lambda x: str(x)) 

budgetPD['ProdBudget'] = budgetPD.ProdBudget.apply(lambda x: int(x.replace('$','').replace(',',''))) 

budgetPD['DGross'] = budgetPD.DGross.apply(lambda x: int(x.replace('$','').replace(',',''))) 

budgetPD['WwGross'] = budgetPD.WwGross.apply(lambda x: int(x.replace('$','').replace(',','')))

In [ ]:
# del n1

In [ ]:
budgetPD.sort_values(by=['WwGross'], ascending=False)
budgetPD2 = budgetPD[budgetPD['DGross']>0]
budgetPD2

In [ ]:
len(budgetPD2.ReleaseDate.sort_values()[245:])

In [ ]:
budgetPD2.ReleaseDate.sort_values()[245:]

In [ ]:
np.where(budgetPD2.mvName=='Finding Dory')

In [ ]:
budgetPD2.iloc[34,:]

In [ ]:
budgetPD2.iloc[34,:].ProdBudget

In [ ]:
# test=budgetPD2
# test

In [ ]:
test.dtypes

In [ ]:
# test['mvName']=test['mvName'].astype('str')

In [ ]:
test.dtypes

In [ ]:
# test.join(df2, on=['mvName'], how='left', lsuffix='r_') ## doesn't work

In [ ]:
mv_listDF

In [ ]:
mojopgDF=pd.read_html('http://www.boxofficemojo.com/movies/?id=pixar2015.htm')

In [ ]:
mojopgDF=pd.read_html('http://www.boxofficemojo.com/movies/?id=junglebook2015.htm')

In [ ]:
mojopgDF[4]

In [ ]:
mojopgDF[5][0][3][13:]

In [ ]:
"".join((mojopgDF[5][1][2][8:].split('hrs.')[0].replace(' ','') , ':', mojopgDF[5][1][2][8:].split('hrs.')[1].replace('min.','').replace(' ','') ))

In [ ]:
# testtim = "".join((mojopgDF[5][1][2][8:].split('hrs.')[0].replace(' ','') , ':', mojopgDF[5][1][2][8:].split('hrs.')[1].replace('min.','').replace(' ','') ,':00'))

In [ ]:
# (datetime.datetime.strptime(testtim,'%H:%M:%S'))

In [ ]:
H,Min = (int(mojopgDF[5][1][2][8:].split('hrs.')[0].replace(' ','')) , int(mojopgDF[5][1][2][8:].split('hrs.')[1].replace('min.','').replace(' ','') )  )

In [ ]:
H*60+Min

In [ ]:
mojopgDF[5]

In [ ]:
mojopgDF[9].shape , mojopgDF[10].shape

In [ ]:
GrossInfo = mojopgDF[10]
GrossInfo

In [ ]:
# dGross 
# GrossInfo[1][0].replace('$','').replace(',','')

In [ ]:
# ForeignGross 
GrossInfo[1][1].replace('$','').replace(',','')

In [ ]:
# TotalGross 
GrossInfo[1][3].replace('$','').replace(',','')

In [ ]:
# mojopgDF[10] #Opening Weekend:	$135,060,273

In [ ]:
# mojopgDF[11] #Widest Release:	4,305 theaters

In [ ]:
mojopgDF[12] 
mojopgDF[14]

In [ ]:
mojopgDF[13]

In [ ]:
mojopgDF[13][1][0].split('(co-director)')

In [ ]:
mojopgDF[13][1][3]

In [ ]:
mojopgDF[14]